<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/nlp/lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import copy
docs=["Cute kitty","Eat rice or cake","Kitty and hamster","Eat bread","Rice, bread and cake","Cute hamster eats bread and cake"]
docs

['Cute kitty',
 'Eat rice or cake',
 'Kitty and hamster',
 'Eat bread',
 'Rice, bread and cake',
 'Cute hamster eats bread and cake']

In [ ]:
docs_1=["Cute kitty","Eat rice cake","Kitty hamster","Eat bread","Rice bread cake","Cute hamster eat bread cake"]

In [ ]:
doc=[x.lower().split() for x in docs_1]
doc

[['cute', 'kitty'],
 ['eat', 'rice', 'cake'],
 ['kitty', 'hamster'],
 ['eat', 'bread'],
 ['rice', 'bread', 'cake'],
 ['cute', 'hamster', 'eat', 'bread', 'cake']]

In [ ]:
class LDA:
    def __init__(self,token_ls):
        self.token_ls=token_ls
        self.word_ls=sorted(list(set(sum(token_ls,[]))))
        self.doc_word_index=[[self.word_ls.index(y) for y in x] for x in self.token_ls]

    def lda(self,topic_num,alpha,beta,epoch_num):
        # np.random.seed(0)
        # token_j=[list(np.random.randint(low=0,high=topic_num,size=len(x))) for x in self.token_ls]
        token_j=[[0,1],[0,1,0],[1,1],[0,0],[0,1,0],[1,1,1,0,0]]# 문장의 단어당 주제 할당
        token_index_ls=[(i,j) for i,x in enumerate(self.token_ls) for j,_ in enumerate(x) ]
        for n in range(epoch_num):
            token_j_copy=copy.deepcopy(token_j)
            for p,q in token_index_ls:
                token_j_copy[p][q]=np.nan
                token_j_copy=[np.array(x) for x in token_j_copy]

                doc_j=np.array([[(x==i).sum() for x in token_j_copy] for i in range(topic_num)]) # 문서당 주제 count

                token_j_index=[[(i,j) for i,_ in enumerate(self.token_ls) for j,_ in enumerate(self.token_ls[i]) if self.token_ls[i][j]==x] for x in self.word_ls]
                word_j=np.array([[(np.array([token_j_copy[x[0]][x[1]] for x in v])==i).sum() for v in token_j_index]for i in range(topic_num)])

                doc_prob=(doc_j+alpha)/(doc_j+alpha).sum(axis=0)
                word_prob=(word_j+beta).T/(word_j+beta).sum(axis=1)

                token_j_copy[p][q]=np.argmax([doc_prob[i][p]*word_prob.T[i][self.word_ls.index(self.token_ls[p][q])] for i in range(topic_num)])
                token_j_copy=[list(x) for x in token_j_copy]

            if token_j==token_j_copy:
                    break
            token_j=token_j_copy
            return token_j_copy

In [ ]:
LDA(doc).lda(topic_num=2,alpha=0.1,beta=0.001,epoch_num=20)

[[1.0, 1.0],
 [0.0, 0.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0]]